# General Saccade Analysis

This notebook demonstrates how to detect and characterize saccades (rapid turns)
in fly trajectories, independent of any stimulus events.

## What you'll learn

1. Detecting saccades using angular velocity thresholds
2. Analyzing saccade kinematics
3. Comparing left vs right turns
4. Visualizing saccade distributions

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import braidz_analysis as ba

print(f"braidz_analysis version: {ba.__version__}")

## 1. Load Data

In [ ]:
# === CONFIGURE YOUR DATA PATH HERE ===
DATA_PATH = "/path/to/your/experiments"  # Update this!
BRAIDZ_FILE = "experiment.braidz"  # Update this!

# Load data
data = ba.read_braidz(BRAIDZ_FILE, base_folder=DATA_PATH)
print(data)

## 2. Analyze All Saccades

Detect all saccades across all trajectories.

In [ ]:
# Analyze saccades with default parameters
saccades = ba.analyze_saccades(
    data.trajectories,
    flight_only=True,  # Only analyze during flight (not walking)
    progressbar=True,
)

print(saccades)
print(f"\nTotal saccades detected: {len(saccades)}")

In [ ]:
# Examine the results structure
print("Traces available:")
for key, arr in saccades.traces.items():
    print(f"  {key}: {arr.shape}")

print("\nMetrics columns:")
print(saccades.metrics.columns.tolist())

In [ ]:
# Preview saccade metrics
saccades.metrics.head(10)

## 3. Saccade Kinematics Overview

In [ ]:
# Summary statistics
print("Saccade Statistics:")
print("=" * 50)

hc = saccades.metrics["heading_change"]
pv = saccades.metrics["peak_velocity"]

print("Heading change (absolute):")
print(f"  Mean: {np.degrees(np.abs(hc).mean()):.1f} deg")
print(f"  Median: {np.degrees(np.abs(hc).median()):.1f} deg")
print(f"  Std: {np.degrees(np.abs(hc).std()):.1f} deg")

print("\nPeak angular velocity (absolute):")
print(f"  Mean: {np.degrees(np.abs(pv).mean()):.1f} deg/s")
print(f"  Median: {np.degrees(np.abs(pv).median()):.1f} deg/s")
print(f"  Max: {np.degrees(np.abs(pv).max()):.1f} deg/s")

print("\nTurn direction:")
n_left = (saccades.metrics["direction"] == 1).sum()
n_right = (saccades.metrics["direction"] == -1).sum()
print(f"  Left (CCW): {n_left} ({n_left/len(saccades)*100:.1f}%)")
print(f"  Right (CW): {n_right} ({n_right/len(saccades)*100:.1f}%)")

## 4. Visualizing Saccade Traces

In [ ]:
# Mean angular velocity around saccades
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# All saccades (absolute value)
ba.plot_angular_velocity(
    saccades,
    ax=axes[0],
    use_abs=True,
    baseline_range=None,
    stimulus_range=None,  # No stimulus for general saccades
)
axes[0].axvline(
    50, color="gray", linestyle="--", alpha=0.5, label="Saccade peak"
)  # Assuming pre_frames=50
axes[0].legend()
axes[0].set_title("Angular Velocity (Absolute)")

# Linear velocity
ba.plot_linear_velocity(
    saccades,
    ax=axes[1],
    stimulus_range=None,
)
axes[1].axvline(50, color="gray", linestyle="--", alpha=0.5)
axes[1].set_title("Linear Velocity")

plt.tight_layout()
plt.show()

In [ ]:
# Compare left vs right turns
left_turns = saccades.left_turns
right_turns = saccades.right_turns

print(f"Left turns: {len(left_turns)}")
print(f"Right turns: {len(right_turns)}")

fig, ax = plt.subplots(figsize=(8, 4))

# Plot raw angular velocity (not absolute) to show direction
ba.plot_traces(
    left_turns, ax=ax, use_abs=False, convert_to_degrees=True, color="tab:blue", label="Left (CCW)"
)
ba.plot_traces(
    right_turns, ax=ax, use_abs=False, convert_to_degrees=True, color="tab:red", label="Right (CW)"
)

ax.axvline(50, color="gray", linestyle="--", alpha=0.5)
ax.axhline(0, color="black", linestyle="-", alpha=0.3)
ax.legend()
ax.set_title("Angular Velocity by Turn Direction")
plt.show()

## 5. Heading Change Distribution

In [ ]:
# Heading change histogram
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Standard histogram
ba.plot_heading_distribution(saccades, ax=axes[0], convert_to_degrees=True)
axes[0].set_title("Heading Change Distribution")

# Polar histogram
ax_polar = plt.subplot(1, 2, 2, projection="polar")
ba.plot_heading_distribution(saccades, ax=ax_polar, polar=True)
ax_polar.set_title("Heading Change (Polar)")

plt.tight_layout()
plt.show()

In [ ]:
# Compare left vs right with violin plot
fig, ax = plt.subplots(figsize=(8, 4))

ba.plot_heading_comparison(
    groups=["Left Turns", "Right Turns"], results_list=[left_turns, right_turns], ax=ax
)
ax.set_title("Heading Change by Turn Direction")
plt.show()

## 6. Saccade Relationships

Explore relationships between saccade properties.

In [ ]:
# Heading change vs peak velocity
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

hc_deg = np.degrees(saccades.metrics["heading_change"])
pv_deg = np.degrees(saccades.metrics["peak_velocity"])

# Scatter plot
ax = axes[0]
ax.scatter(np.abs(hc_deg), np.abs(pv_deg), alpha=0.3, s=10)
ax.set_xlabel("|Heading Change| (deg)")
ax.set_ylabel("|Peak Angular Velocity| (deg/s)")
ax.set_title("Heading Change vs Peak Velocity")

# Add correlation
from scipy import stats

r, p = stats.pearsonr(np.abs(hc_deg), np.abs(pv_deg))
ax.annotate(
    f"r = {r:.2f}\np < 0.001" if p < 0.001 else f"r = {r:.2f}\np = {p:.3f}",
    xy=(0.05, 0.95),
    xycoords="axes fraction",
    va="top",
)

# Histogram of peak velocities
ax = axes[1]
ax.hist(np.abs(pv_deg), bins=50, alpha=0.7, edgecolor="black")
ax.set_xlabel("|Peak Angular Velocity| (deg/s)")
ax.set_ylabel("Count")
ax.set_title("Distribution of Peak Velocities")
ax.axvline(300, color="red", linestyle="--", label="Detection threshold (300 deg/s)")
ax.legend()

plt.tight_layout()
plt.show()

## 7. Individual Saccade Visualization

In [ ]:
# Find the largest saccade
idx_max = np.argmax(np.abs(saccades.metrics["heading_change"]))

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Trajectory
ba.plot_trajectory(
    saccades,
    index=idx_max,
    dims=("x", "y"),
    ax=axes[0],
    highlight_range=(45, 55),  # Around peak
    normalize=True,
)
axes[0].set_title("Trajectory (X-Y)")

# Angular velocity
ax = axes[1]
omega = np.degrees(saccades.traces["angular_velocity"][idx_max])
ax.plot(omega)
ax.axvline(50, color="red", linestyle="--", alpha=0.5, label="Peak")
ax.axhline(0, color="gray", linestyle="-", alpha=0.3)
ax.set_xlabel("Frames")
ax.set_ylabel("Angular Velocity (deg/s)")
ax.set_title("Angular Velocity Trace")
ax.legend()

# Linear velocity
ax = axes[2]
speed = saccades.traces["linear_velocity"][idx_max]
ax.plot(speed)
ax.axvline(50, color="red", linestyle="--", alpha=0.5)
ax.set_xlabel("Frames")
ax.set_ylabel("Linear Velocity (m/s)")
ax.set_title("Linear Velocity Trace")

hc = np.degrees(saccades.metrics["heading_change"].iloc[idx_max])
pv = np.degrees(saccades.metrics["peak_velocity"].iloc[idx_max])
fig.suptitle(f"Largest Saccade: Heading Change = {hc:.1f}°, Peak Velocity = {pv:.1f}°/s")

plt.tight_layout()
plt.show()

## 8. Custom Detection Parameters

Adjust saccade detection sensitivity.

In [ ]:
# Compare different thresholds
thresholds = [200, 300, 400, 500]
results = []

for thresh in thresholds:
    config = ba.Config(saccade_threshold=thresh)
    res = ba.analyze_saccades(data.trajectories, config=config, progressbar=False)
    results.append(res)
    print(f"Threshold {thresh} deg/s: {len(res)} saccades detected")

In [ ]:
# Visualize effect of threshold
fig, ax = plt.subplots(figsize=(8, 5))

counts = [len(r) for r in results]
ax.bar(range(len(thresholds)), counts, tick_label=[str(t) for t in thresholds])
ax.set_xlabel("Detection Threshold (deg/s)")
ax.set_ylabel("Number of Saccades Detected")
ax.set_title("Effect of Detection Threshold on Saccade Count")

plt.tight_layout()
plt.show()

## 9. Per-Trajectory Statistics

In [ ]:
# Count saccades per trajectory
saccades_per_traj = saccades.metrics.groupby("obj_id").size()

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Histogram
ax = axes[0]
ax.hist(saccades_per_traj, bins=30, alpha=0.7, edgecolor="black")
ax.set_xlabel("Saccades per Trajectory")
ax.set_ylabel("Count")
ax.set_title("Distribution of Saccade Counts")
ax.axvline(
    saccades_per_traj.mean(),
    color="red",
    linestyle="--",
    label=f"Mean = {saccades_per_traj.mean():.1f}",
)
ax.legend()

# Summary stats
ax = axes[1]
ax.boxplot(saccades_per_traj)
ax.set_ylabel("Saccades per Trajectory")
ax.set_title("Saccade Count Distribution")

print("Saccades per trajectory:")
print(f"  Mean: {saccades_per_traj.mean():.1f}")
print(f"  Median: {saccades_per_traj.median():.1f}")
print(f"  Min: {saccades_per_traj.min()}")
print(f"  Max: {saccades_per_traj.max()}")

plt.tight_layout()
plt.show()

## Summary

In this notebook, you learned how to:

1. Detect saccades using `ba.analyze_saccades()`
2. Examine saccade kinematics (heading change, peak velocity)
3. Filter by turn direction using `.left_turns` and `.right_turns`
4. Visualize saccade distributions with histograms and polar plots
5. Explore relationships between saccade properties
6. Adjust detection parameters with `ba.Config`
7. Compute per-trajectory statistics

**Key parameters for saccade detection:**
- `saccade_threshold`: Minimum angular velocity to detect (deg/s)
- `min_saccade_spacing`: Minimum frames between saccades
- `flight_only`: Only detect during flight (recommended)